In [38]:
!nvidia-smi

Thu Apr 24 05:27:05 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 570.124.06             Driver Version: 570.124.06     CUDA Version: 12.8     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA L4                      Off |   00000000:00:03.0 Off |                    0 |
| N/A   76C    P0             35W /   72W |    2529MiB /  23034MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [39]:
!apt update
!apt install libcudnn9-cuda-12
!pip install nvidia-cublas-cu12 nvidia-cudnn-cu12==9.*

!export LD_LIBRARY_PATH=`python3 -c 'import os; import nvidia.cublas.lib; import nvidia.cudnn.lib; print(os.path.dirname(nvidia.cublas.lib.__file__) + ":" + os.path.dirname(nvidia.cudnn.lib.__file__))'`


Hit:1 https://packages.cloud.google.com/apt gcsfuse-jammy InRelease
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,605 kB]
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:6 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Get:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Hit:12 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Fetched 1,863 kB in 2s (917 kB/s)
Reading package lists... Done
Building dependency tree.

In [40]:
!pip3 install faster-whisper ctranslate2 datasets sacrebleu -q

In [41]:
from datasets import load_dataset, Dataset, Audio
import numpy as np

In [42]:
import os
cache_dir = "/content/huggingface_cache"
os.makedirs(cache_dir, exist_ok=True)

# Set ALL Hugging Face related cache directories
os.environ["TRANSFORMERS_CACHE"] = os.path.join(cache_dir, "transformers")
os.environ["HF_DATASETS_CACHE"] = os.path.join(cache_dir, "datasets")
os.environ["HF_HOME"] = os.path.join(cache_dir, "hf_home")
os.environ["HF_ASSETS_CACHE"] = os.path.join(cache_dir, "assets")
os.environ["HUGGINGFACE_HUB_CACHE"] = os.path.join(cache_dir, "hub")
os.environ["HF_MODULES_CACHE"] = os.path.join(cache_dir, "modules")

# Create all directories
for dir_path in [os.environ["TRANSFORMERS_CACHE"],
                os.environ["HF_DATASETS_CACHE"],
                os.environ["HF_HOME"],
                os.environ["HF_ASSETS_CACHE"],
                os.environ["HUGGINGFACE_HUB_CACHE"],
                os.environ["HF_MODULES_CACHE"]]:
    os.makedirs(dir_path, exist_ok=True)

# Force datasets to use the new cache
from datasets import config
config.HF_DATASETS_CACHE = os.environ["HF_DATASETS_CACHE"]

# Load the Dataset

In [43]:
dataset_name = "kreasof-ai/bem-eng-IWSLT2025"
test_dataset = load_dataset(dataset_name, trust_remote_code=True, split="test")

Resolving data files:   0%|          | 0/41 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/41 [00:00<?, ?it/s]

In [44]:
test_dataset

Dataset({
    features: ['audio', 'sentence', 'translation', 'speaker_id'],
    num_rows: 2763
})

In [45]:
for sample in test_dataset:
    print(sample["audio"])  # Access the audio column
    break  # Just to check the first example

{'path': '15_7474_3_152_02_211030-180248_bem_8cc_elicit_1.wav', 'array': array([ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
       -9.15527344e-05, -6.10351562e-05, -6.10351562e-05]), 'sampling_rate': 16000}


# Create Arrays for audio

In [46]:
references = test_dataset["translation"]
print(references) # Check if the references is list

['he is wearing glasses as well', 'A dog with a wide nose is running on the lawns of the football ground', 'She has her leg in the air attempting to break a board.', 'There are also small vehicles that they use in cold places behind them.', 'Of the men on the roof of the house, ine is wearing a long sleeved grey shirt and a blue trousers.', 'These on their shoes are for sliding when the to play on the ice.', 'A woman is walking in the road with a child on her showders.', "I have seen it seems there are different games being played that's why others are looking the other way while others are watching volleyball.", 'A woman is holding food on a plate inside a shop.', 'Or maybe someone has made him upset.', 'four people are inside an area with large rocks and they are wearing helmets', "Another child's shoes being taken off by the instructor.", 'Maybe he wants to reach the top of the mountain. The rode metal he is carrying, it is mostly used when one is climbing the mountains.', 'There is

In [47]:
audio_arrays = [a["array"].astype(np.float32) for a in test_dataset["audio"]]

In [48]:

# audio_arrays

In [49]:
assert len(audio_arrays) == len(references)

In [50]:
len(audio_arrays)

2763

In [51]:
len(references)

2763

# Cascaded

## transcribe

In [52]:
# model_id = "kreasof-ai/whisper-medium-bem2en"
# output_dir="ct2-whisper-medium-transcription-finetuned"
# commit_hash= "2b91ce20bd264d43947d18db44d7d08e84ae49ee"

# !ct2-transformers-converter \
# --model {model_id} \
# --output_dir {output_dir} \
# --revision {commit_hash} \
# --quantization float16 \
# --copy_files tokenizer_config.json\
# --force

In [53]:
model_id = "openai/whisper-small"
output_dir="ct2-whisper-small-transcription-baseline"

!ct2-transformers-converter \
--model {model_id} \
--output_dir {output_dir} \
--quantization float16 \
--copy_files tokenizer_config.json\
--force

/usr/local/lib/python3.10/dist-packages/transformers/utils/hub.py:128: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(
2025-04-24 05:27:41.726099: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-04-24 05:27:42.167097: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-04-24 05:27:42.327677: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-04-24 05:27:42.369529: E external/local_xla/xla/stream_executor/cuda/c

In [54]:
from faster_whisper import WhisperModel

model_name = output_dir

model = WhisperModel(model_name, device="cuda", compute_type="float16")

In [55]:
test_dataset[0]

{'audio': {'path': '15_7474_3_152_02_211030-180248_bem_8cc_elicit_1.wav',
  'array': array([ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
         -9.15527344e-05, -6.10351562e-05, -6.10351562e-05]),
  'sampling_rate': 16000},
 'sentence': 'nafwala na amakalashi ku menso',
 'translation': 'he is wearing glasses as well',
 'speaker_id': 152}

In [56]:
tgt_lang = "en"

transcriptions = []
segments, info = model.transcribe(audio_arrays[0],
                                beam_size=5,
                                language=tgt_lang,
                                vad_filter=True
                                )
transcription = " ".join([segment.text.strip() for segment in segments])
transcriptions.append(transcription)

In [57]:
print(transcriptions)

["Now, let's start the process."]


In [58]:
from tqdm.auto import tqdm

tgt_lang = "en"

transcriptions = []

for audio_array in tqdm(audio_arrays, total=len(audio_arrays)):
    segments, info = model.transcribe(audio_array,
                                        beam_size=5,
                                        language=tgt_lang,
                                        vad_filter=True)
    transcription = " ".join([segment.text.strip() for segment in segments])
    transcriptions.append(transcription)

  0%|          | 0/2763 [00:00<?, ?it/s]

In [59]:
print(*transcriptions[:20], sep="\n")

Now, let's start the process.
If you don't have earphones, you don't have to buy a new one.
This is the end of this video. Thank you for watching.
This is what I'm going to show you. This is what I'm going to show you. This is what I'm going to show you.
Awao mewari pamutenge yangan da umo afuere ishatsu yami to mito Iriyama boku ayatari ero ne toroshi Iriyama kumbi makumbi
If you've ever been to Kunsapatu, if you're a Kuttelelele, you might want to come and visit the place.
Namayu Arendamumusebo Nabi Kanumwana Pamawea
That's it, that's it. It's time to say goodbye. I hope you enjoy watching this video. And I'll see you in the next video.
Now we are going to dry the bamboo shoots and cut them into pieces.
Nangulimbi, gulivamu avamu furuise.
I will see you in the next video.
Come on, I can't be better than full of amazapato-kuri and full of shawava.
I'm going to make a fire so that it doesn't burn. I'm going to make a fire so that it doesn't burn.
I hope that you liked this video. I hop

## MT (NLLB)

In [60]:
# mt_model_name = "kreasof-ai/nllb-200-distilled-600M-bem2en-flores200"
# output_mt = "ct2-nllb-be-en-finetuned"
# commit_hash = "ede0247483834828669a0a44366bd1763d176733"
# !ct2-transformers-converter --model {mt_model_name} --revision {commit_hash} --quantization float16 --output_dir {output_mt} --force

In [ ]:
mt_model_name = "facebook/nllb-200-3.3B"
output_mt = "ct2-nllb-bem-eng-baseline"
!ct2-transformers-converter --model {mt_model_name} --quantization float16 --output_dir {output_mt} --force

/usr/local/lib/python3.10/dist-packages/transformers/utils/hub.py:128: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(
2025-04-24 06:23:43.745843: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-04-24 06:23:44.202094: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-04-24 06:23:44.359757: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-04-24 06:23:44.407696: E external/local_xla/xla/stream_executor/cuda/c

In [72]:
# !sudo rm -rf ct2-nllb-be-en-baseline/

In [63]:
# !wget https://s3.amazonaws.com/opennmt-models/nllb-200/flores200_sacrebleu_tokenizer_spm.model

In [ ]:
import os

directory = "/content/"

ct2_model_name = output_mt

ct_model_path = os.path.join(directory, ct2_model_name)
sp_model_path = os.path.join(directory, "flores200_sacrebleu_tokenizer_spm.model")

In [ ]:
import ctranslate2
import sentencepiece as spm
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"

sp = spm.SentencePieceProcessor()
sp.load(sp_model_path)

translator = ctranslate2.Translator(ct_model_path, device=device)

In [ ]:
# Translate

# src_lang = "ben_Beng"
# tgt_lang = "eng_Latn"

src_lang = "bem_Latn"
tgt_lang = "eng_Latn"

beam_size = 5

source_sentences = transcriptions

source_sents = [sent.strip() for sent in source_sentences]
target_prefix = [[tgt_lang]] * len(source_sents)

# Subword the source sentences
source_sents_subworded = sp.encode_as_pieces(source_sents)
source_sents_subworded = [[src_lang] + sent + ["</s>"] for sent in source_sents_subworded]

# Translate the source sente nces
translations = translator.translate_batch(source_sents_subworded,
                                          batch_type="tokens",
                                          max_batch_size=2024,
                                          beam_size=beam_size,
                                          target_prefix=target_prefix)
translations = [translation.hypotheses[0] for translation in translations]

# Desubword the target sentences
translations_desubword = sp.decode(translations)
translations_desubword = [sent[len(tgt_lang):].strip() for sent in translations_desubword]

print(*translations_desubword[:10], sep="\n")

In [ ]:
import sacrebleu

translations = translations_desubword

# Calculate BLEU
bleu = sacrebleu.corpus_bleu(translations, [references])
bleu = round(bleu.score, 2)

# Calculate CHRF
chrf = sacrebleu.corpus_chrf(translations, [references], word_order=2)  # for chrF++ word_order=2
chrf = round(chrf.score, 2)

print(model_name)
print("BLEU\tChrF++")
print(f"{bleu}\t{chrf}")

# Comet

In [ ]:
!pip install -U unbabel-comet -q

In [ ]:
from comet import download_model, load_from_checkpoint

In [ ]:
lst_data = [{"src": src, "mt":mt, "ref":ref} for src, mt, ref in zip(source_sentences, translations_desubword, references)]

In [ ]:
model_path = download_model("masakhane/africomet-mtl")
model = load_from_checkpoint(model_path)
data = lst_data
model_output = model.predict(data, batch_size=8, gpus=1)
print (model_output)

In [ ]:
sys_score = model_output["system_score"]

In [ ]:
print("COMET:", round(sys_score*100, 2))

In [ ]:
sys_score